In [3]:
pip install pandas scikit-learn xgboost joblib tqdm


In [4]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
import joblib
import zipfile

# Upload kaggle.json file before this step
from google.colab import files
uploaded = files.upload()

# ✅ Set up custom Kaggle config path
print("🔑 Setting up Kaggle API...")
os.makedirs("/content/.kaggle", exist_ok=True)
with open("/content/.kaggle/kaggle.json", "wb") as f:
    f.write(uploaded["kaggle.json"])
os.environ["KAGGLE_CONFIG_DIR"] = "/content/.kaggle"

# ✅ Download dataset
print("⬇️ Downloading dataset...")
!kaggle datasets download -d sulianova/cardiovascular-disease-dataset

# ✅ Unzip dataset
print("🗂️ Unzipping dataset...")
with zipfile.ZipFile("cardiovascular-disease-dataset.zip", "r") as zip_ref:
    zip_ref.extractall("data")

# ✅ Load dataset
print("📄 Loading dataset...")
df = pd.read_csv("data/cardio_train.csv", sep=";")

# ✅ Preprocessing
print("🧼 Preprocessing...")
df["age"] = (df["age"] / 365).astype(int)
df = df[(df["ap_hi"] > 0) & (df["ap_hi"] < 300)]
df = df[(df["ap_lo"] > 0) & (df["ap_lo"] < 200)]
df = df[df["ap_hi"] >= df["ap_lo"]]
df.drop("id", axis=1, inplace=True)

cat_cols = ["gender", "cholesterol", "gluc", "smoke", "alco", "active"]
num_cols = ["age", "height", "weight", "ap_hi", "ap_lo"]

for col in tqdm(cat_cols, desc="🔤 Encoding categorical"):
    df[col] = LabelEncoder().fit_transform(df[col])

scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

X = df.drop("cardio", axis=1)
y = df["cardio"]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# ✅ Train Random Forest
print("🌲 Training RandomForest...")
rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
rf.fit(X_train, y_train)
print(f"✅ RF Acc: {accuracy_score(y_val, rf.predict(X_val)):.4f} | ROC-AUC: {roc_auc_score(y_val, rf.predict_proba(X_val)[:,1]):.4f}")

# ✅ Train XGBoost
print("⚡ Training XGBoost...")
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb.fit(X_train, y_train)
print(f"✅ XGB Acc: {accuracy_score(y_val, xgb.predict(X_val)):.4f} | ROC-AUC: {roc_auc_score(y_val, xgb.predict_proba(X_val)[:,1]):.4f}")

# ✅ Train Neural Network
print("🧠 Training Neural Network...")
nn = MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=300, early_stopping=True, random_state=42)
nn.fit(X_train, y_train)
print(f"✅ NN Acc: {accuracy_score(y_val, nn.predict(X_val)):.4f} | ROC-AUC: {roc_auc_score(y_val, nn.predict_proba(X_val)[:,1]):.4f}")

# Save best model (XGBoost)
joblib.dump(xgb, "best_model.pkl")

# ✅ Prediction function
def predict_from_input(input_dict):
    try:
        input_df = pd.DataFrame([input_dict])
        input_df[num_cols] = scaler.transform(input_df[num_cols])
        for col in cat_cols:
            input_df[col] = LabelEncoder().fit_transform([input_dict[col]])[0]
        input_df = input_df[X.columns.tolist()]
        pred = xgb.predict(input_df)[0]
        return f"🔍 Prediction: {'HIGH risk' if pred else 'LOW risk'}"
    except Exception as e:
        return f"❌ Prediction error: {e}"

# ✅ Sample prediction
sample = {
    "age": 50, "height": 170, "weight": 70, "ap_hi": 120, "ap_lo": 80,
    "gender": 1, "cholesterol": 1, "gluc": 1, "smoke": 0, "alco": 0, "active": 1
}
print("📦 Sample prediction:")
print(predict_from_input(sample))


Saving kaggle.json to kaggle.json
🔑 Setting up Kaggle API...
⬇️ Downloading dataset...
Dataset URL: https://www.kaggle.com/datasets/sulianova/cardiovascular-disease-dataset
License(s): unknown
  0% 0.00/742k [00:00<?, ?B/s]
100% 742k/742k [00:00<00:00, 264MB/s]
🗂️ Unzipping dataset...
📄 Loading dataset...
🧼 Preprocessing...


🔤 Encoding categorical: 100%|██████████| 6/6 [00:00<00:00, 258.54it/s]

🌲 Training RandomForest...


✅ RF Acc: 0.7279 | ROC-AUC: 0.7981
⚡ Training XGBoost...


/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [13:08:35] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


✅ XGB Acc: 0.7296 | ROC-AUC: 0.7955
🧠 Training Neural Network...
✅ NN Acc: 0.7290 | ROC-AUC: 0.7955
📦 Sample prediction:
🔍 Prediction: LOW risk


In [5]:
# STEP 0: Install dependencies
!pip install -q kaggle xgboost tqdm scikit-learn

# STEP 1: Upload your kaggle.json manually or ensure it exists in the runtime
import os, zipfile
from google.colab import files
uploaded = files.upload()

os.makedirs("/root/.kaggle", exist_ok=True)
!mv kaggle.json /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

# STEP 2: Download dataset from Kaggle
print("🔑 Setting up Kaggle API...")
!kaggle datasets download -d sulianova/cardiovascular-disease-dataset

# STEP 3: Unzip dataset
print("🗂️ Unzipping dataset...")
with zipfile.ZipFile("cardiovascular-disease-dataset.zip", "r") as zip_ref:
    zip_ref.extractall("/content")

# STEP 4: Import and preprocess
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from tqdm import tqdm

print("📄 Loading dataset...")
df = pd.read_csv("/content/cardio_train.csv", sep=';')
df.drop(columns=['id'], inplace=True)

print("🧼 Preprocessing...")
# Upsample minority class
df_major = df[df.cardio == 0]
df_minor = df[df.cardio == 1]
df_minor_upsampled = resample(df_minor, replace=True, n_samples=len(df_major), random_state=42)
df = pd.concat([df_major, df_minor_upsampled]).sample(frac=1, random_state=42)

# Features and labels
X = df.drop("cardio", axis=1)
y = df["cardio"]

# Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Models
print("🌲 Training RandomForest...")
rf = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)

print("⚡ Training XGBoost...")
xgb = XGBClassifier(n_estimators=200, max_depth=5, learning_rate=0.05, use_label_encoder=False, eval_metric="logloss")
xgb.fit(X_train, y_train)
xgb_pred = xgb.predict(X_test)

print("🧠 Training Neural Network...")
mlp = MLPClassifier(hidden_layer_sizes=(128, 64), activation='relu', max_iter=300, random_state=42)
mlp.fit(X_train, y_train)
mlp_pred = mlp.predict(X_test)

# Voting Ensemble
print("🧠 Combining all (Voting Classifier)...")
ensemble = VotingClassifier(estimators=[
    ('rf', rf), ('xgb', xgb), ('mlp', mlp)
], voting='soft')

ensemble.fit(X_train, y_train)
ensemble_pred = ensemble.predict(X_test)

# Metrics
acc = accuracy_score(y_test, ensemble_pred)
roc = roc_auc_score(y_test, ensemble_pred)
cm = confusion_matrix(y_test, ensemble_pred)

print(f"\n✅ Ensemble Accuracy: {acc:.4f}")
print(f"✅ ROC-AUC Score: {roc:.4f}")
print(f"🧾 Confusion Matrix:\n{cm}")


Saving kaggle.json to kaggle (1).json
🔑 Setting up Kaggle API...
Dataset URL: https://www.kaggle.com/datasets/sulianova/cardiovascular-disease-dataset
License(s): unknown
cardiovascular-disease-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
🗂️ Unzipping dataset...
📄 Loading dataset...
🧼 Preprocessing...
🌲 Training RandomForest...
⚡ Training XGBoost...


/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [13:11:01] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


🧠 Training Neural Network...


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


🧠 Combining all (Voting Classifier)...


/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [13:15:56] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(



✅ Ensemble Accuracy: 0.7468
✅ ROC-AUC Score: 0.7468
🧾 Confusion Matrix:
[[5497 1509]
 [2038 4965]]


In [6]:
# -------------------- SETUP --------------------
print("🔧 Installing & importing dependencies...")
!pip install -q kaggle tqdm xgboost scikit-learn pandas matplotlib seaborn

import os
import zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score
from sklearn.neural_network import MLPClassifier
import xgboost as xgb

# -------------------- AUTH & DOWNLOAD --------------------
print("🔑 Setting up Kaggle API...")
os.makedirs("/root/.kaggle", exist_ok=True)
!mv kaggle.json /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

print("⬇️ Downloading dataset...")
!kaggle datasets download -d sulianova/cardiovascular-disease-dataset
with zipfile.ZipFile("cardiovascular-disease-dataset.zip", "r") as zip_ref:
    zip_ref.extractall("data")

# -------------------- LOAD & PREPROCESS --------------------
print("📄 Loading dataset...")
df = pd.read_csv("data/cardio_train.csv", sep=";")

print("🧼 Preprocessing...")
df.drop("id", axis=1, inplace=True)
df["age"] = df["age"] // 365  # Convert age from days to years

# Remove outliers (height and weight beyond quantile thresholds)
df = df[(df["height"] > df["height"].quantile(0.01)) & (df["height"] < df["height"].quantile(0.99))]
df = df[(df["weight"] > df["weight"].quantile(0.01)) & (df["weight"] < df["weight"].quantile(0.99))]

X = df.drop("cardio", axis=1)
y = df["cardio"]

print("📊 Splitting dataset...")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# -------------------- TRAIN MODELS --------------------
print("🌲 Training RandomForest...")
rf = RandomForestClassifier(n_estimators=150, max_depth=12, random_state=42)
rf.fit(X_train, y_train)

print("⚡ Training XGBoost...")
xgb_model = xgb.XGBClassifier(n_estimators=150, learning_rate=0.08, random_state=42)
xgb_model.fit(X_train, y_train)

print("🧠 Training NeuralNet...")
mlp = MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=300, random_state=42)
mlp.fit(X_train, y_train)

# -------------------- ENSEMBLE --------------------
print("🧠 Combining all (Voting Classifier)...")
ensemble = VotingClassifier(estimators=[
    ('rf', rf),
    ('xgb', xgb_model),
    ('nn', mlp)
], voting='soft')

ensemble.fit(X_train, y_train)

# -------------------- EVALUATE --------------------
print("📈 Evaluating ensemble model...")
y_pred = ensemble.predict(X_test)
acc = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, ensemble.predict_proba(X_test)[:, 1])
cm = confusion_matrix(y_test, y_pred)

print(f"\n✅ Ensemble Accuracy: {acc:.4f}")
print(f"✅ ROC-AUC Score: {roc_auc:.4f}")
print("🧾 Confusion Matrix:")
print(cm)

# -------------------- TEST SAMPLE --------------------
def predict_user_input(model, input_dict):
    input_df = pd.DataFrame([input_dict])
    return model.predict(input_df)[0]

sample = {
    "age": 50, "gender": 1, "height": 165, "weight": 72,
    "ap_hi": 120, "ap_lo": 80, "cholesterol": 1,
    "gluc": 1, "smoke": 0, "alco": 0, "active": 1
}

print("\n📦 Sample prediction:")
result = predict_user_input(ensemble, sample)
print("🔍 Prediction:", "HIGH risk" if result == 1 else "LOW risk")


🔧 Installing & importing dependencies...
🔑 Setting up Kaggle API...
mv: cannot stat 'kaggle.json': No such file or directory
⬇️ Downloading dataset...
Dataset URL: https://www.kaggle.com/datasets/sulianova/cardiovascular-disease-dataset
License(s): unknown
cardiovascular-disease-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
📄 Loading dataset...
🧼 Preprocessing...
📊 Splitting dataset...
🌲 Training RandomForest...
⚡ Training XGBoost...
🧠 Training NeuralNet...
🧠 Combining all (Voting Classifier)...
📈 Evaluating ensemble model...

✅ Ensemble Accuracy: 0.7368
✅ ROC-AUC Score: 0.8036
🧾 Confusion Matrix:
[[5234 1500]
 [2025 4633]]

📦 Sample prediction:
🔍 Prediction: LOW risk


In [7]:
# -------------------- SETUP --------------------
print("🔧 Installing dependencies...")
!pip install -q kagglehub tqdm xgboost scikit-learn pandas matplotlib seaborn

import kagglehub
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score
import xgboost as xgb

# -------------------- DOWNLOAD DATA --------------------
print("⬇️ Downloading dataset from KaggleHub...")
dataset_path = kagglehub.dataset_download("sulianova/cardiovascular-disease-dataset")
print("📁 Path to dataset files:", dataset_path)

# -------------------- LOAD & CLEAN --------------------
print("📄 Loading dataset...")
df = pd.read_csv(f"{dataset_path}/cardio_train.csv", sep=";")
df.drop("id", axis=1, inplace=True)
df["age"] = df["age"] // 365  # convert days to years

print("🧼 Cleaning and preprocessing...")
# Remove outliers
df = df[(df["height"] > df["height"].quantile(0.01)) & (df["height"] < df["height"].quantile(0.99))]
df = df[(df["weight"] > df["weight"].quantile(0.01)) & (df["weight"] < df["weight"].quantile(0.99))]

# Split data
X = df.drop("cardio", axis=1)
y = df["cardio"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# -------------------- TRAIN MODELS --------------------
print("🌲 Training RandomForest...")
rf = RandomForestClassifier(n_estimators=150, max_depth=12, random_state=42)
rf.fit(X_train, y_train)

print("⚡ Training XGBoost...")
xgb_model = xgb.XGBClassifier(n_estimators=150, learning_rate=0.08, random_state=42)
xgb_model.fit(X_train, y_train)

print("🧠 Training Neural Network...")
mlp = MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=300, random_state=42)
mlp.fit(X_train, y_train)

# -------------------- ENSEMBLE --------------------
print("🧠 Combining all models with VotingClassifier...")
ensemble = VotingClassifier(estimators=[
    ('rf', rf),
    ('xgb', xgb_model),
    ('mlp', mlp)
], voting='soft')

ensemble.fit(X_train, y_train)

# -------------------- EVALUATE --------------------
print("📊 Evaluating ensemble...")
y_pred = ensemble.predict(X_test)
acc = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, ensemble.predict_proba(X_test)[:, 1])
cm = confusion_matrix(y_test, y_pred)

print(f"\n✅ Ensemble Accuracy: {acc:.4f}")
print(f"✅ ROC-AUC Score: {roc_auc:.4f}")
print("🧾 Confusion Matrix:")
print(cm)

# -------------------- PREDICT SAMPLE --------------------
def predict_sample(model, features: dict):
    df_input = pd.DataFrame([features])
    prediction = model.predict(df_input)[0]
    return "HIGH risk" if prediction == 1 else "LOW risk"

sample = {
    "age": 50, "gender": 1, "height": 165, "weight": 72,
    "ap_hi": 120, "ap_lo": 80, "cholesterol": 1,
    "gluc": 1, "smoke": 0, "alco": 0, "active": 1
}

print("\n🔎 Sample Prediction:")
print("🔍 Risk level:", predict_sample(ensemble, sample))


🔧 Installing dependencies...
⬇️ Downloading dataset from KaggleHub...


100%|██████████| 742k/742k [00:00<00:00, 78.5MB/s]

Extracting files...
📁 Path to dataset files: /root/.cache/kagglehub/datasets/sulianova/cardiovascular-disease-dataset/versions/1
📄 Loading dataset...
🧼 Cleaning and preprocessing...


🌲 Training RandomForest...
⚡ Training XGBoost...
🧠 Training Neural Network...
🧠 Combining all models with VotingClassifier...
📊 Evaluating ensemble...

✅ Ensemble Accuracy: 0.7368
✅ ROC-AUC Score: 0.8036
🧾 Confusion Matrix:
[[5234 1500]
 [2025 4633]]

🔎 Sample Prediction:
🔍 Risk level: LOW risk


In [8]:
# 🔧 Install dependencies
!pip install -q kagglehub tqdm imbalanced-learn xgboost

# 📦 Imports
import pandas as pd
import numpy as np
import kagglehub
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE

# 🔄 Load dataset
print("⬇️ Downloading dataset from KaggleHub...")
path = kagglehub.dataset_download("sulianova/cardiovascular-disease-dataset")
print("📁 Path to dataset files:", path)

# 📄 Load CSV
df = pd.read_csv("/kaggle/input/cardiovascular-disease-dataset/cardio_train.csv", sep=";")

# 🧼 Preprocess
print("🧼 Cleaning and preprocessing...")
if 'id' in df.columns:
    df.drop('id', axis=1, inplace=True)

X = df.drop('cardio', axis=1)
y = df['cardio']

# ⚖️ Balance data
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X, y)

# 🎯 Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=42)

# 🧪 Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 🧠 Models
print("🌲 Training RandomForest...")
rf = RandomForestClassifier(n_estimators=300, max_depth=15, random_state=42)

print("⚡ Training XGBoost...")
xgb = XGBClassifier(n_estimators=300, max_depth=7, learning_rate=0.05, subsample=0.8,
                    use_label_encoder=False, eval_metric='logloss', random_state=42)

print("🧠 Training Neural Network...")
mlp = MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=400, alpha=0.0005, solver='adam', random_state=42)

# 🤝 Voting Classifier
print("🧠 Combining all models with VotingClassifier...")
ensemble = VotingClassifier(estimators=[
    ('rf', rf),
    ('xgb', xgb),
    ('mlp', mlp)
], voting='soft')

# 📈 Fit model with tqdm
print("🏋️ Training Ensemble...")
for _ in tqdm(range(1), desc="Training"):
    ensemble.fit(X_train_scaled, y_train)

# 🎯 Evaluate
y_pred = ensemble.predict(X_test_scaled)
acc = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, ensemble.predict_proba(X_test_scaled)[:, 1])
conf_matrix = confusion_matrix(y_test, y_pred)

print("\n✅ Ensemble Accuracy:", round(acc, 4))
print("✅ ROC-AUC Score:", round(roc_auc, 4))
print("🧾 Confusion Matrix:\n", conf_matrix)

# 🔍 Prediction sample
sample = X_test.iloc[0:1]
risk = ensemble.predict(scaler.transform(sample))[0]
print("\n🔎 Sample Prediction:")
print("🔍 Risk level:", "HIGH risk" if risk else "LOW risk")


⬇️ Downloading dataset from KaggleHub...
📁 Path to dataset files: /kaggle/input/cardiovascular-disease-dataset
🧼 Cleaning and preprocessing...
🌲 Training RandomForest...
⚡ Training XGBoost...
🧠 Training Neural Network...
🧠 Combining all models with VotingClassifier...
🏋️ Training Ensemble...


Training:   0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [13:27:23] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
Training: 100%|██████████| 1/1 [06:11<00:00, 371.66s/it]



✅ Ensemble Accuracy: 0.7412
✅ ROC-AUC Score: 0.8025
🧾 Confusion Matrix:
 [[5372 1552]
 [2074 5011]]

🔎 Sample Prediction:
🔍 Risk level: HIGH risk


In [9]:
# ✅ Full Improved Cardiovascular Disease Classification Pipeline
# ✅ Includes: Scaling, Outlier Handling, SMOTE, RF+XGB+MLP, Stacking, EarlyStopping, TQDM

# ---- STEP 1: Install Required Libraries ----
!pip install -q kaggle kagglehub xgboost lightgbm imbalanced-learn tqdm

# ---- STEP 2: Import Libraries ----
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from imblearn.over_sampling import SMOTE
from tqdm.notebook import tqdm
import kagglehub

# ---- STEP 3: Download Dataset ----
print("⬇️ Downloading dataset from KaggleHub...")
path = kagglehub.dataset_download("sulianova/cardiovascular-disease-dataset")
print("📁 Path to dataset files:", path)

# ---- STEP 4: Load Data ----
df = pd.read_csv(f"{path}/cardio_train.csv", sep=';')
df.drop(columns=['id'], inplace=True)

# ---- STEP 5: Clean & Engineer Features ----
print("🧼 Cleaning and preprocessing...")
# Remove outliers
q_low, q_hi = df.quantile(0.01), df.quantile(0.99)
for col in ['ap_hi', 'ap_lo', 'weight', 'height']:
    df = df[(df[col] >= q_low[col]) & (df[col] <= q_hi[col])]

# Fix invalid blood pressure
df = df[df['ap_hi'] > df['ap_lo']]

# Feature engineering
df['bmi'] = df['weight'] / ((df['height']/100)**2)

# ---- STEP 6: Split Features ----
X = df.drop(columns=['cardio'])
y = df['cardio']

# Normalize features
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)

# ---- STEP 7: Train-Test Split ----
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, stratify=y, test_size=0.2, random_state=42)

# ---- STEP 8: Balance Data ----
print("🧪 Applying SMOTE to balance classes...")
sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

# ---- STEP 9: Initialize Models ----
rf = RandomForestClassifier(n_estimators=100, random_state=42)
xgb = XGBClassifier(n_estimators=100, use_label_encoder=False, eval_metric='logloss', random_state=42)
mlp = MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=300, early_stopping=True, random_state=42)

# ---- STEP 10: Build Ensemble (Stacking) ----
print("🧠 Combining all models with StackingClassifier...")
ensemble = StackingClassifier(
    estimators=[
        ('rf', rf),
        ('xgb', xgb),
        ('mlp', mlp)
    ],
    final_estimator=LogisticRegression(),
    passthrough=True,
    cv=5,
    n_jobs=-1
)

# ---- STEP 11: Train Model ----
print("🏋️ Training Ensemble...")
for _ in tqdm(range(1), desc="Training"):
    ensemble.fit(X_train_res, y_train_res)

# ---- STEP 12: Evaluation ----
y_pred = ensemble.predict(X_test)
y_prob = ensemble.predict_proba(X_test)[:, 1]

acc = accuracy_score(y_test, y_pred)
roc = roc_auc_score(y_test, y_prob)
cm = confusion_matrix(y_test, y_pred)

print(f"\n✅ Ensemble Accuracy: {acc:.4f}")
print(f"✅ ROC-AUC Score: {roc:.4f}")
print("🧾 Confusion Matrix:")
print(cm)

# ---- STEP 13: Sample Prediction ----
sample = X_test[0].reshape(1, -1)
sample_pred = ensemble.predict(sample)[0]
print("\n🔎 Sample Prediction:")
print("🔍 Risk level:", "HIGH risk" if sample_pred else "LOW risk")


⬇️ Downloading dataset from KaggleHub...
📁 Path to dataset files: /kaggle/input/cardiovascular-disease-dataset
🧼 Cleaning and preprocessing...
🧪 Applying SMOTE to balance classes...
🧠 Combining all models with StackingClassifier...
🏋️ Training Ensemble...


Training:   0%|          | 0/1 [00:00<?, ?it/s]


✅ Ensemble Accuracy: 0.7308
✅ ROC-AUC Score: 0.7976
🧾 Confusion Matrix:
[[5253 1422]
 [2123 4372]]

🔎 Sample Prediction:
🔍 Risk level: HIGH risk


In [10]:
!pip install -q imbalanced-learn


In [11]:
# 🔧 Install dependencies
!pip install -q kagglehub imbalanced-learn xgboost tqdm

# 📚 Imports
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from tqdm import tqdm
import kagglehub

# ⬇️ Download dataset using KaggleHub
print("⬇️ Downloading dataset from KaggleHub...")
path = kagglehub.dataset_download("sulianova/cardiovascular-disease-dataset")
print("📁 Path to dataset files:", path)

# 📄 Load the dataset
print("📄 Loading dataset...")
df = pd.read_csv(os.path.join(path, "cardio_train.csv"), sep=';')

# 🧼 Preprocessing
print("🧼 Cleaning and preprocessing...")
df.drop(columns=["id"], inplace=True)
df["age"] = (df["age"] / 365).astype(int)

# Separate features and target
X = df.drop("cardio", axis=1)
y = df["cardio"]

# 🧪 Apply SMOTE
print("🧪 Applying SMOTE to balance classes...")
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# 🔀 Split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# 🔨 Define models
print("⚙️ Defining base models...")
rf = RandomForestClassifier(n_estimators=150, max_depth=20, random_state=42, n_jobs=-1)
xgb = XGBClassifier(n_estimators=150, learning_rate=0.05, max_depth=5, use_label_encoder=False, eval_metric='logloss')
mlp = MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=300, random_state=42)

# 🧠 Combine models using Stacking
print("🧠 Combining all models with StackingClassifier...")
estimators = [
    ("rf", rf),
    ("xgb", xgb),
    ("mlp", mlp)
]
stacked_model = StackingClassifier(estimators=estimators, final_estimator=RandomForestClassifier(n_estimators=100, random_state=42))

# 🏋️ Training
print("🏋️ Training Ensemble...")
for _ in tqdm(range(1), desc="Training"):
    stacked_model.fit(X_train, y_train)

# 📈 Evaluation
y_pred = stacked_model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
roc = roc_auc_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

print(f"\n✅ Ensemble Accuracy: {acc:.4f}")
print(f"✅ ROC-AUC Score: {roc:.4f}")
print("🧾 Confusion Matrix:")
print(cm)

# 🔍 Prediction sample
sample_input = {
    "age": 50, "gender": 1, "height": 165, "weight": 70, "ap_hi": 120,
    "ap_lo": 80, "cholesterol": 1, "gluc": 1, "smoke": 0, "alco": 0, "active": 1
}
sample_df = pd.DataFrame([sample_input])
sample_pred = stacked_model.predict(sample_df)[0]
print("\n🔎 Sample Prediction:")
print("🔍 Risk level:", "HIGH risk" if sample_pred == 1 else "LOW risk")


⬇️ Downloading dataset from KaggleHub...
📁 Path to dataset files: /kaggle/input/cardiovascular-disease-dataset
📄 Loading dataset...
🧼 Cleaning and preprocessing...
🧪 Applying SMOTE to balance classes...
⚙️ Defining base models...
🧠 Combining all models with StackingClassifier...
🏋️ Training Ensemble...


Training:   0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [13:35:57] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [13:37:28] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [13:37:29] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [13:37:29] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.11/dist-packages/xgboost/train


✅ Ensemble Accuracy: 0.7176
✅ ROC-AUC Score: 0.7178
🧾 Confusion Matrix:
[[5110 1814]
 [2142 4943]]

🔎 Sample Prediction:
🔍 Risk level: LOW risk


In [12]:
!pip install kagglehub lightgbm xgboost imbalanced-learn tqdm


In [13]:
# 🔧 Install dependencies
!pip install -q kagglehub lightgbm xgboost imbalanced-learn tqdm

# 📚 Imports
import pandas as pd
import numpy as np
import kagglehub
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from sklearn.ensemble import StackingClassifier
from imblearn.over_sampling import SMOTE
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings("ignore")

# ⬇️ Download dataset from KaggleHub
print("⬇️ Downloading dataset from KaggleHub...")
path = kagglehub.dataset_download("sulianova/cardiovascular-disease-dataset")
print("📁 Path to dataset files:", path)

# 📄 Load dataset
print("📄 Loading dataset...")
df = pd.read_csv(f"{path}/cardio_train.csv", sep=';')
df.drop('id', axis=1, inplace=True)

# 🧼 Preprocessing
print("🧼 Cleaning and preprocessing...")
df = df[(df['ap_hi'] > 0) & (df['ap_lo'] > 0)]
df = df[(df['ap_hi'] < 250) & (df['ap_lo'] < 200)]
df['age'] = (df['age'] / 365).astype(int)  # convert days to years

X = df.drop("cardio", axis=1)
y = df["cardio"]

# 🧪 Apply SMOTE to balance classes
print("🧪 Applying SMOTE to balance classes...")
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X, y)

# 📊 Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_res, y_res, test_size=0.2, random_state=42, stratify=y_res
)

# 🔄 Feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# ⚙️ Define base models
print("⚙️ Defining base models...")
base_models = [
    ('rf', RandomForestClassifier(n_estimators=150, max_depth=12, random_state=42)),
    ('xgb', XGBClassifier(n_estimators=100, max_depth=6, learning_rate=0.1, use_label_encoder=False, eval_metric='logloss')),
    ('lgb', LGBMClassifier(n_estimators=120, max_depth=6, learning_rate=0.1))
]

# 🧠 Define meta learner
meta_model = MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=300, random_state=42)

# 🧠 Create stacking ensemble
print("🧠 Combining all models with StackingClassifier...")
stack = StackingClassifier(estimators=base_models, final_estimator=meta_model, passthrough=True)

# 🏋️ Train ensemble with tqdm
print("🏋️ Training Ensemble...")
for _ in tqdm(range(1), desc="Training"):
    stack.fit(X_train, y_train)

# 📈 Predict and evaluate
y_pred = stack.predict(X_test)
y_proba = stack.predict_proba(X_test)[:, 1]

acc = accuracy_score(y_test, y_pred)
roc = roc_auc_score(y_test, y_proba)
cm = confusion_matrix(y_test, y_pred)

print(f"\n✅ Ensemble Accuracy: {acc:.4f}")
print(f"✅ ROC-AUC Score: {roc:.4f}")
print(f"🧾 Confusion Matrix:\n{cm}")

# 🔍 Sample prediction
print("\n🔎 Sample Prediction:")
sample = X_test[0].reshape(1, -1)
sample_pred = stack.predict(sample)[0]
risk = "HIGH" if sample_pred else "LOW"
print(f"🔍 Risk level: {risk} risk")


⬇️ Downloading dataset from KaggleHub...
📁 Path to dataset files: /kaggle/input/cardiovascular-disease-dataset
📄 Loading dataset...
🧼 Cleaning and preprocessing...
🧪 Applying SMOTE to balance classes...
⚙️ Defining base models...
🧠 Combining all models with StackingClassifier...
🏋️ Training Ensemble...


Training:   0%|          | 0/1 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 27875, number of negative: 27875
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005698 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 545
[LightGBM] [Info] Number of data points in the train set: 55750, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posi